In [206]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.preprocessing import StandardScaler

In [207]:
pd.options.display.max_columns = 150

In [208]:
df = pd.read_csv('kc_house_data_clean.csv')
print(df.shape)
df.describe()

(21596, 21)


,Unnamed: 0,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,year,corr_zipcode
count,21596.000000,2.159600e+04,21596.000000,21596.000000,21596.000000,2.159600e+04,21596.000000,21596.000000,21596.000000,21596.000000,21596.000000,21596.000000,21596.000000,21596.000000,21596.000000,21596.000000,21596.000000,21596.000000,21596.000000,21596.000000,21596.000000
mean,10807.368402,5.401983e+05,3.371828,2.115843,2080.343165,1.509983e+04,1.494119,0.007548,0.234303,3.409752,7.657946,1788.631506,291.711660,1971.000787,84.468698,47.560087,-122.213977,1986.650722,12758.656649,2014.322976,0.127477
std,6239.633454,3.671416e+05,0.904114,0.768998,918.122038,4.141355e+04,0.539685,0.086551,0.766406,0.650471,1.173218,827.763251,442.673703,29.375460,401.830330,0.138552,0.140725,685.231768,27275.018316,0.467625,0.333515
min,0.000000,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,0.000000,1900.000000,0.000000,47.155900,-122.519000,399.000000,651.000000,2014.000000,0.000000
25%,5403.750000,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,47.471100,-122.328000,1490.000000,5100.000000,2014.000000,0.000000
50%,10809.500000,4.500000e+05,3.000000,2.250000,1910.000000,7.619000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,47.571800,-122.231000,1840.000000,7620.000000,2014.000000,0.000000
75%,16211.250000,6.450000e+05,4.000000,2.500000,2550.000000,1.068550e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,47.678000,-122.125000,2360.000000,10083.000000,2015.000000,0.000000
max,21612.000000,7.700000e+06,11.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,47.777600,-121.315000,6210.000000,871200.000000,2015.000000,1.000000


In [209]:
df.head(5)

,Unnamed: 0,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,year,corr_zipcode
0,0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,47.5112,-122.257,1340,5650,2014,0
1,1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,47.7210,-122.319,1690,7639,2014,0
2,2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,47.7379,-122.233,2720,8062,2015,0
3,3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,47.5208,-122.393,1360,5000,2014,0
4,4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,47.6168,-122.045,1800,7503,2015,0


In [210]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21596 entries, 0 to 21595
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     21596 non-null  int64  
 1   price          21596 non-null  float64
 2   bedrooms       21596 non-null  int64  
 3   bathrooms      21596 non-null  float64
 4   sqft_living    21596 non-null  int64  
 5   sqft_lot       21596 non-null  int64  
 6   floors         21596 non-null  float64
 7   waterfront     21596 non-null  int64  
 8   view           21596 non-null  int64  
 9   condition      21596 non-null  int64  
 10  grade          21596 non-null  int64  
 11  sqft_above     21596 non-null  int64  
 12  sqft_basement  21596 non-null  int64  
 13  yr_built       21596 non-null  int64  
 14  yr_renovated   21596 non-null  int64  
 15  lat            21596 non-null  float64
 16  long           21596 non-null  float64
 17  sqft_living15  21596 non-null  int64  
 18  sqft_l

In [211]:
df.corrwith(df.price)

Unnamed: 0       0.027449
price            1.000000
bedrooms         0.315966
bathrooms        0.525915
sqft_living      0.701921
sqft_lot         0.089885
floors           0.256831
waterfront       0.266438
view             0.397323
condition        0.036000
grade            0.667935
sqft_above       0.605395
sqft_basement    0.323772
yr_built         0.053995
yr_renovated     0.126418
lat              0.306772
long             0.022103
sqft_living15    0.585273
sqft_lot15       0.082841
year             0.003757
corr_zipcode     0.449995
dtype: float64

In [212]:
y = df["price"]
X = df.drop("price", axis=1)


In [213]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape); print(X_test.shape)
print(y.shape)

(17276, 20)
(4320, 20)
(21596,)


In [214]:
# Transformation
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X),columns=X.columns)

In [215]:
# Modèle de Regression Linéaire
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures


model = make_pipeline(PolynomialFeatures(2), LinearRegression())

model.fit(X_train,y_train)
model.score(X_test,y_test)*100



83.27988029273395

In [216]:
# Méthode de Regression de Ridge
rr = Ridge(alpha=0.01)

rr.fit(X_train, y_train) 

pred_train_rr= rr.predict(X_train)

print(np.sqrt(mean_squared_error(y_train,pred_train_rr)))

print(r2_score(y_train, pred_train_rr)*100)


pred_test_rr= rr.predict(X_test)

print(np.sqrt(mean_squared_error(y_test,pred_test_rr))) 

print(r2_score(y_test, pred_test_rr)*100)

191418.22047419476
73.457482817411
176104.4452443515
74.50828173810784


In [217]:
# Modèle Lasso
model_lasso = Lasso(alpha=0.01)

model_lasso.fit(X_train, y_train) 

pred_train_lasso= model_lasso.predict(X_train)

print(np.sqrt(mean_squared_error(y_train,pred_train_lasso)))

print(r2_score(y_train, pred_train_lasso)*100)


pred_test_lasso= model_lasso.predict(X_test)

print(np.sqrt(mean_squared_error(y_test,pred_test_lasso))) 

print(r2_score(y_test, pred_test_lasso)*100)

191418.22041572817
73.45748283362524
176104.34987535272
74.50830934807517


/home/kamel/miniconda3/envs/kbp/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.898e+14, tolerance: 2.385e+11
  model = cd_fast.enet_coordinate_descent(


In [218]:
# Modèle Elastic Net

model_enet = ElasticNet(alpha = 0.01)

model_enet.fit(X_train, y_train) 

pred_train_enet= model_enet.predict(X_train)

print(np.sqrt(mean_squared_error(y_train,pred_train_enet)))

print(r2_score(y_train, pred_train_enet)*100)


pred_test_enet= model_enet.predict(X_test)

print(np.sqrt(mean_squared_error(y_test,pred_test_enet)))

print(r2_score(y_test, pred_test_enet)*100)

193105.60102568043
72.98746767778289
177973.85759519413
73.96420157086223


/home/kamel/miniconda3/envs/kbp/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.367e+14, tolerance: 2.385e+11
  model = cd_fast.enet_coordinate_descent(
